In [1]:
%pip install -U google.generativeai # Install the Python SDK

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.generativeai as genai
import os
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

/home/alex/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Get text from a file
def get_text_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()
def write_text_to_file(file_path, text):
    with open(file_path, 'w') as file:
        file.write(text)
def get_command_output(command, folder=None):
    if folder:
        # enter the folder first
        os.chdir(folder)
    output = os.popen(command).read()
    if folder:
        # return to the previous folder
        os.chdir('..')
    return output

def get_test_case_output(executable, input_type, input_file, folder):
        if input_type == "filein":
            return get_command_output("./" + executable + " < " + input_file, folder)
        elif input_type == "pythonin":
            return get_command_output("python3 " + input_file + " " + executable, folder)
        else:
            print("Invalid input type")
            return None

print(get_command_output('pwd'))
print(get_command_output('pwd', 'vulnerable1'))
print(get_command_output('pwd', 'samples'))


/home/alex/code/safe-code

/home/alex/code/safe-code/vulnerable1

/home/alex/code/safe-code/samples



In [28]:
# TEST CASES
test_suite = []

# Test cases for basic.c:
test_case_1 = {"folder": "vulnerable1", "file": "basic.c", "exe": "basic.exe", "test_cases": []}
test_case_1["test_cases"].append({"input-type": "filein", "input": "in1", "correct": True})
# a test case is valid if ./{executable} < in1 is the same for both executable (c.exe and rust.exe)
test_case_1["test_cases"].append({"input-type": "pythonin", "input": "attack.py", "correct": False})
# a test case is valid if ./{executable} < (python3 attack.py) has different output as out1
test_suite.append(test_case_1)

def run_test_cases(generate_rust_code):
    # Get the performance of the model
    for test in test_suite:
        print(f"Running test cases for {test['folder']}/{test['file']}")
        # First make sure the c file is there
        if not "exe" in test or not os.path.exists(test["folder"] + "/" + test["exe"]):
            print("No executable found")
            continue

        # Generate the rust code for the given input
        input_c_code = get_text_from_file(test["folder"] + "/" + test["file"])
        rust_code = generate_rust_code(input_c_code)
        # create a file for the rust code, rust.rs
        write_text_to_file(test["folder"] + "/rust.rs", rust_code)
        # create the executable rust.exe
        get_command_output("rustc rust.rs -o rust.exe", test["folder"])

        for test_number, test_case in enumerate(test["test_cases"]):
            c_output = get_test_case_output(test["exe"], test_case["input-type"], test_case["input"], test["folder"])
            rust_output = get_test_case_output("rust.exe", test_case["input-type"], test_case["input"], test["folder"])
            print(f"C Output({len(c_output)}): {c_output}")
            print(f"Rust Output({len(rust_output)}): {rust_output}")
            if test_case["correct"]:
                if c_output == rust_output:
                    print(f"Test case {test_number} passed (correct result)")
                else:
                    print(f"Test case {test_number} failed (incorrect result)")
            else:
                if c_output != rust_output:
                    print(f"Test case {test_number} passed (protected against attack)")
                else:
                    print(f"Test case {test_number} failed (failed to protect against attack)")

In [30]:
model = genai.GenerativeModel('gemini-pro')
def generate_rust_code(c_code):
    response = model.generate_content("Translate the following C code to Rust:\n\n" + c_code)
    # remove the "```rust" first line and the last "```"
    print(f"(DEBUG) Translated code: {response.text}")
    rust_code = response.text.split("\n")[1:-1]
    return "\n".join(rust_code)
run_test_cases(generate_rust_code)

Running test cases for vulnerable1/basic.c
(DEBUG) Translated code: ```rust
use std::io::{self, Write};

#[derive(Debug)]
struct Student {
    name: String,
    grade: String,
}

fn main() {
    let mut student = Student {
        name: String::new(),
        grade: "nil".to_string(),
    };

    print!("Enter your name: ");
    io::stdout().flush().expect("Could not flush stdout");
    io::stdin().read_line(&mut student.name).expect("Could not read line");

    println!("Hi {}! Your grade is {}.", student.name, student.grade);
}
```
C Output(28): Hi Alex! Your grade is nil.

Rust Output(45): Enter your name: Hi Alex! Your grade is nil.

Test case 0 failed (incorrect result)
C Output(28): Hi Alex! Your grade is A+.


Rust Output(55): Enter your name: Hi Alex      A+ ! Your grade is nil.


Test case 1 passed (protected against attack)


In [31]:
c_code = get_text_from_file("snake/snake.c")
response = model.generate_content("Translate the following C code to Rust:\n\n" + c_code)
print(response.text)

KeyboardInterrupt: 